# Data Types: Counters

In [8]:
import riak

client = riak.RiakClient()

In [9]:
client.is_alive()

True

http://docs.basho.com/riak/kv/2.2.3/developing/data-types/

http://docs.basho.com/riak/kv/2.2.3/developing/data-types/counters/#create-a-counter

In [45]:
%alias riak-admin docker exec -it compose-riak_coordinator_1 riak-admin

In [48]:
%riak-admin bucket-type create maps '{"props":{"datatype":"map"}}'
%riak-admin bucket-type create sets '{"props":{"datatype":"set"}}'
%riak-admin bucket-type create counters '{"props":{"datatype":"counter"}}'

maps created

can no longer be downgraded to a version of Riak prior to 2.0
sets created

can no longer be downgraded to a version of Riak prior to 2.0
counters created

can no longer be downgraded to a version of Riak prior to 2.0
hlls created

can no longer be downgraded to a version of Riak prior to 2.0


In [50]:
%riak-admin bucket-type status counters

counters has been created and may be activated

young_vclock: 20
w: quorum
small_vclock: 50
rw: quorum
r: quorum
pw: 0
precommit: []
pr: 0
postcommit: []
old_vclock: 86400
notfound_ok: true
n_val: 3
linkfun: {modfun,riak_kv_wm_link_walker,mapreduce_linkfun}
last_write_wins: false
dw: quorum
dvv_enabled: true
chash_keyfun: {riak_core_util,chash_std_keyfun}
big_vclock: 50
basic_quorum: false
allow_mult: true
datatype: counter
active: false
claimant: 'riak@172.20.0.2'


In [58]:
%riak-admin bucket-type activate counters

counters has been activated

downgraded to a version of Riak prior to 2.0


## Create a counter

To create a counter, you need to specify a bucket/key pair to hold that counter. Here is the general syntax for doing so:

In [59]:
bucket = client.bucket_type('counters').bucket('traffic_tickets')

Let’s say that we want to create a counter called traffic_tickets in our counters bucket to keep track of our legal misbehavior. We can create this counter and ensure that the counters bucket will use our counters bucket type like this:

In [60]:
counter = bucket.new('traffic_tickets')

## Increment a counter

Now that our client knows which bucket/key pairing to use for our counter, traffic_tickets will start out at 0 by default. If we happen to get a ticket that afternoon, we can increment the counter:

In [61]:
counter.increment()

# Updates are staged locally and have to be explicitly sent to Riak
# using the store() method.
counter.store()

## Increment a counter with more that one

The default value of an increment operation is 1, but you can increment by more than 1 (but always by an integer).

Continuing with our traffic_tickets example, let’s say we receive 5 tickets in a single day:

In [62]:
counter.increment(5)
counter.store()

## Retrieve a counter value

We can retrieve the value of the counter and view how many tickets have accumulated:

In [65]:
# To fetch the value stored on the server, use the call below. Note
# that this will clear any changes to the counter that have not yet been
# sent to Riak
counter.reload()

print("Value:", counter.value)

Value: 2


## Decrement a counter value

Counters enable you to decrement values in addition to incrementing them as seen above.

For example, let’s say we hire an expert lawyer who gets one of the traffic tickets stricken from our record:

In [66]:
counter.decrement()

# Just like incrementing, you can also decrement by more than one, e.g.:
counter.decrement(3)
counter.store()

## Retrieve a counter value with curl

In [67]:
!curl http://localhost:8098/types/counters/buckets/traffic_tickets/datatypes/traffic_tickets

{"type":"counter","value":-2}